In [1]:
import sys
print(sys.version)

3.7.16 (default, Jan 17 2023, 16:06:28) [MSC v.1916 64 bit (AMD64)]


In [ ]:
!pip install tensorflow==2.3.1
!pip install keras_applications==1.0.8
!pip install keras==2.4.3
!pip install keras-vggface==0.6
!pip install joblib
!pip install matplotlib
!pip install mtcnn
!pip install scikit-learn

In [46]:
import os
import joblib
import cv2
import numpy as np
import matplotlib.pyplot as plt
import mtcnn
from keras_vggface.vggface import VGGFace
from tensorflow.keras.preprocessing import image
from keras_vggface.utils import preprocess_input

In [15]:
mtcnn_model=mtcnn.MTCNN()

In [31]:
model=VGGFace(model='resnet50',include_top=False,input_shape=(224,224,3),pooling='avg')

In [38]:

def face_embbeder(img):
    individual_face=[]
    face_box=mtcnn_model.detect_faces(img)
    
    for i in range(len(face_box)):
        x1,y1,w,h=face_box[i]['box']
        cropped_face=img[y1:y1+h,x1:x1+w]
        cropped_face=cv2.resize(cropped_face,(224,224))
        cropped_face=image.img_to_array(cropped_face)
        cropped_face=np.expand_dims(cropped_face,axis=0)
        cropped_face=preprocess_input(cropped_face)
        emb=model.predict(cropped_face)
        individual_face.append(emb)
        
    return individual_face

In [42]:
faces_dict={}

for i in os.listdir('images'):
    img_name='images/'+i
    img=cv2.imread(img_name)
    faces_dict[img_name]=face_embbeder(img)

In [48]:
file=open('faces_embedding','wb')
joblib.dump(faces_dict,file)